In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext
import time


/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
list_city_dt = []

for file_in in os.listdir(r'../../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()
print(list_city_dt)
print(len(list_city_dt))


['berlin_db=0.50', 'berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00', 'frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00', 'paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']
16


In [4]:

list_workforce_xlsx = []

for base_file in list_city_dt:

    base_file_xlsx = f'{base_file}_all_trials.xlsx'
    if base_file_xlsx in os.listdir(r'../../workforce_size'):
        df_ = pd.read_excel(f'../../workforce_size/{base_file_xlsx}')
        list_workforce_xlsx.append(df_)


In [7]:
df_workforce = pd.concat(list_workforce_xlsx, ignore_index = True)

df_workforce.columns = ['instance', 'city', 'DB', 'OC',
       'RM', 'GM', 'model', 'max_n_shifts',
       'workforce_size', 'wage_costs', 'objective_value',
       'objective_value_post_wage', 'workforce_size_trial']

df_workforce.head(20)

,instance,city,DB,OC,RM,GM,model,max_n_shifts,workforce_size,wage_costs,objective_value,objective_value_post_wage,workforce_size_trial
0,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,8,192.0,3483.66,3291.66,5
1,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,9,216.0,3461.14,3245.14,6
2,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,10,240.0,3439.62,3199.62,7
3,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,11,264.0,3419.38,3155.38,8
4,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,12,288.0,3400.34,3112.34,9
5,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,13,312.0,3382.66,3070.66,10
6,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,14,336.0,3365.58,3029.58,11
7,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,15,360.0,3349.96,2989.96,12
8,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,16,384.0,3335.80,2951.80,13
9,berlin_db=0.50,berlin,0.5,1.2,1.5,0.8,fixed,NaN,17,408.0,3323.16,2915.16,14


In [8]:
df_workforce['count'] = 1
df_workforce['count'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['count'].transform('sum')

df_workforce_count = df_workforce.copy()
df_workforce_count.drop_duplicates(subset = ['instance','OC','model','max_n_shifts'], inplace = True)

df_workforce_count.sort_values(by = ['count'], inplace = True)

In [11]:
df_workforce_count = df_workforce_count[df_workforce_count['count']==1]
df_workforce_count

,instance,city,DB,OC,RM,GM,model,max_n_shifts,workforce_size,wage_costs,objective_value,objective_value_post_wage,workforce_size_trial,count
785,berlin_db=1.00,berlin,1.0,2.5,1.5,0.8,partflex,4.0,18,NaN,NaN,NaN,15,1.0
784,berlin_db=1.00,berlin,1.0,2.5,1.5,0.8,partflex,3.0,18,NaN,NaN,NaN,15,1.0
1559,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,partflex,4.0,73,NaN,NaN,NaN,70,1.0
1558,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,partflex,3.0,73,NaN,NaN,NaN,70,1.0


In [16]:
#check frankfurt

list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find('frank') > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00']


In [17]:
list_frank = []

for base_file in list_city_dt_subset:
    with open(f'../../workforce_size/{base_file}.json') as f:
        data = json.load(f)
        df_ = pd.DataFrame(data)
        list_frank.append(df_)

df_frank = pd.concat(list_frank, ignore_index = True)



In [19]:
df_frank
df_frank.to_excel(r'../code_cm/check_frank_optimal.xlsx', index = False)

In [24]:
list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find('berlin') > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['berlin_db=0.50', 'berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00']


In [25]:
list_ = []

for base_file in list_city_dt_subset:
    with open(f'../../workforce_size/{base_file}.json') as f:
        data = json.load(f)
        df_ = pd.DataFrame(data)
        list_.append(df_)

df_all = pd.concat(list_, ignore_index = True)

df_all.to_excel(r'../code_cm/check_berlin_optimal.xlsx', index = False)
df_all

,instance_file_base,city,demand_baseline,model,max_n_shifts,outsourcing_cost_multiplier,regional_multiplier,global_multiplier,workforce_size_region0,objective_value_prev,objective_value,objective_value_next
0,berlin_db=0.50,berlin,0.5,fixed,NaN,1.2,1.5,0.8,23,NaN,3.269880e+03,3269.880000
1,berlin_db=0.50,berlin,0.5,flex,NaN,1.2,1.5,0.8,21,NaN,3.149780e+03,3149.780000
2,berlin_db=0.50,berlin,0.5,partflex,2.0,1.2,1.5,0.8,20,NaN,3.150220e+03,3150.220000
3,berlin_db=0.50,berlin,0.5,partflex,3.0,1.2,1.5,0.8,20,NaN,3.149980e+03,3149.980000
4,berlin_db=0.50,berlin,0.5,partflex,4.0,1.2,1.5,0.8,21,NaN,3.149780e+03,3149.780000
...,...,...,...,...,...,...,...,...,...,...,...,...
95,berlin_db=4.00,berlin,4.0,fixed,NaN,2.5,1.5,0.8,115,NaN,5.333186e+04,53331.858995
96,berlin_db=4.00,berlin,4.0,flex,NaN,2.5,1.5,0.8,70,NaN,1.000000e+07,NaN
97,berlin_db=4.00,berlin,4.0,partflex,2.0,2.5,1.5,0.8,115,NaN,5.333186e+04,53331.858995
98,berlin_db=4.00,berlin,4.0,partflex,3.0,2.5,1.5,0.8,70,NaN,1.000000e+07,NaN
